# 库导入

In [2]:
# -*- coding: utf-8 -*-
import networkx as nx
import numpy as np
from node2vec import Node2Vec
import tempfile
from pecanpy import pecanpy as node2vec
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, matthews_corrcoef
)
import random
import os
from tqdm import tqdm
from pecanpy.graph import AdjlstGraph
import pandas as pd

# 待调用的函数

In [3]:
# 0. 评价指标
def evaluate_metrics(y_true, y_scores):
    threshold = np.median([s for s in y_scores if s > 0]) if any(y_scores) else 0
    y_pred = [int(score > threshold) for score in y_scores]
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_scores),
        'average_precision': average_precision_score(y_true, y_scores),
        'mcc': matthews_corrcoef(y_true, y_pred)
    }

# 1. Node2Vec节点嵌入向量生成函数
def generate_node2vec_embeddings(G, dimensions=64, walk_length=30, num_walks=200, p=1, q=1):
    """
    利用Node2Vec算法计算图中每个节点的嵌入向量.
    参数:
        G: 输入的 networkx 图
        dimensions: 嵌入向量维度
        walk_length: 每次随机游走的步长
        num_walks: 每个节点开始的随机游走次数
        p: 返回概率参数 (控制随机游走更偏向 BFS 还是 DFS, p<1 趋向于返回原节点, p>1 趋向于远离)
        q: 离开概率参数 (控制随机游走更偏向 BFS 还是 DFS, q>1 趋向于局部, q<1 趋向于探索远邻)
    返回:
        embeddings: 字典 {节点: 向量(numpy.ndarray)}
    """

    # 使用 node2vec 库进行节点嵌入计算 (需要预先 pip install node2vec)
#     node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, 
#                         num_walks=num_walks, p=p, q=q, workers=os.cpu_count()
# )
#     model = node2vec.fit(window=10, min_count=1, batch_words=4)
#     # 提取每个节点的向量表示
#     embeddings = {}
#     for node in G.nodes():
#         try:
#             embeddings[node] = model.wv[node]  # 尝试直接通过节点索引获取向量
#         except KeyError:
#             # 若节点索引需要字符串类型，则进行转换
#             embeddings[node] = model.wv[str(node)]
#     return embeddings


    # 以下为使用pecanpy进行node2vec嵌入的实现，相比于node2vec库，pecanpy提供了更高效的SparseOTF实现，从而加速
    adj_graph = AdjlstGraph()
    for node in G.nodes():
        adj_graph.add_node(str(node))
    for u, v in G.edges():
        adj_graph.add_edge(str(u), str(v), weight=1, directed=False)

    # 构建 SparseOTF
    model = node2vec.SparseOTF(p=p, q=q, workers=os.cpu_count(), verbose=True)
    graph = model.from_adjlst_graph(adj_graph)
    graph.preprocess_transition_probs()
    # 生成节点嵌入矩阵，使用传入参数
    embed_matrix = graph.embed(dim=dimensions, walk_length=walk_length, num_walks=num_walks)
    # **构建节点ID到向量的映射字典**
    embeddings = {}
    for idx, node_id in enumerate(graph.nodes):   # graph.nodes 是字符串ID列表
        # 转回整型作为字典键（如果节点ID本来就是非数字字符串则直接用原值）
        try:
            node_key = int(node_id)
        except ValueError:
            node_key = node_id
        embeddings[node_key] = embed_matrix[idx]
    return embeddings

# 3.嵌入向量组合函数，将两个节点嵌入向量合成为边特征向量
def combine_embeddings(vec_u, vec_v, method='hadamard'):
    """
    将节点 u 和 v 的嵌入向量组合为边的特征向量.
    支持的组合方式:
        - hadamard: 元素对应相乘 (Hadamard 积)
        - average: 向量平均
        - L1: 向量差的绝对值
        - L2: 向量差的平方
    返回:
        edge_feature: numpy.ndarray 类型的边特征向量
    """
    try:
        if method == 'hadamard':
            return vec_u * vec_v
        elif method == 'average':
            return (vec_u + vec_v) / 2
        elif method == 'L1':
            return np.abs(vec_u - vec_v)
        elif method == 'L2':
            return (vec_u - vec_v) ** 2
    except Exception as e:
        print(f"Error combining embeddings: {e}")
        return None

# 4. 预先构建特征
def precompute_structural_features(G, edge_list, structural_features):
    """
    预计算结构特征字典，避免重复计算。
    返回一个字典结构 {(u,v): [cn, jc, aa, pa], ...}
    """
    feature_dict = {}
    neighbors = {n: set(G.neighbors(n)) for n in G.nodes()}
    degrees = dict(G.degree())

    for u, v in tqdm(edge_list, desc="Precomputing structural features"):
        feat = []
        Nu, Nv = neighbors[u], neighbors[v]
        inter = Nu & Nv
        union = Nu | Nv

        if 'CN' in structural_features:
            feat.append(len(inter))
        if 'JC' in structural_features:
            feat.append(len(inter) / len(union) if union else 0.0)
        if 'AA' in structural_features:
            aa = sum(1 / np.log(degrees[w]) for w in inter if degrees[w] > 1)
            feat.append(aa)
        if 'PA' in structural_features:
            feat.append(degrees[u] * degrees[v])
        if 'HI' in structural_features:
            feat.append(abs(degrees[u] - degrees[v]))
        if 'RA' in structural_features:
            # 资源分配指数
            ra = sum(1 / degrees[w] for w in inter) if inter else 0.0
            feat.append(ra)

        feature_dict[(u, v)] = np.array(feat, dtype=float)
    return feature_dict

# 5. 五折交叉验证构建数据集、模型训练与评估
def link_prediction_cross_validation(G, structural_features, embed_method, dimensions, walk_length, num_walks, p, q):
    """
    对给定图进行5折交叉验证的链路预测实验。
    参数:
        G: 原始完整图 (networkx.Graph)
        structural_features: 元组或列表, 指定使用的结构特征, 例如 ('CN','JC','AA','PA')
        use_node2vec: 是否使用 Node2Vec 嵌入特征
        embed_dim: 节点嵌入维度 (仅当 use_node2vec=True 时有效)
        embed_method: 嵌入向量组合方式 ('hadamard', 'average', 'L1', 'L2')
    返回:
        results: 长度为5的列表, 包含每折的测试准确率 (或其他评价指标)
    """
    # 获取原始图中的所有真实边 (正样本边) 列表
    all_positive_edges = list(G.edges())
    kf = KFold(n_splits=5, shuffle=True)
    results = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(all_positive_edges), start=1):
        print(f"\n----- Fold {fold}/5 -----")
        # 划分训练集正样本边和测试集正样本边
        train_pos_edges = [all_positive_edges[i] for i in train_idx]
        test_pos_edges = [all_positive_edges[i] for i in test_idx]
        # 构建训练子图 (将测试正边移除, 避免训练时看到测试边)
        G_train = nx.Graph()
        G_train.add_nodes_from(G.nodes())        # 保留所有节点
        G_train.add_edges_from(train_pos_edges)  # 添加训练正样本边
        # 准备负样本边 (不存在于原图中的节点对)
        existing_edges = set(tuple(sorted(edge)) for edge in G.edges())
        train_neg_edges = []
        test_neg_edges = []
        train_neg_set = set()
        test_neg_set = set()
        nodes = list(G.nodes())
        # 生成测试负样本 (数量与测试正样本相同)
        while len(test_neg_edges) < len(test_pos_edges):
            u = random.choice(nodes); v = random.choice(nodes)
            if u == v:
                continue
            pair = tuple(sorted((u, v)))
            if pair in existing_edges or pair in test_neg_set:
                continue
            test_neg_set.add(pair)
            test_neg_edges.append((u, v))
        # 生成训练负样本 (数量与训练正样本相同)
        while len(train_neg_edges) < len(train_pos_edges):
            u = random.choice(nodes); v = random.choice(nodes)
            if u == v:
                continue
            pair = tuple(sorted((u, v)))
            if pair in existing_edges or pair in train_neg_set or pair in test_neg_set:
                continue
            train_neg_set.add(pair)
            train_neg_edges.append((u, v))
        # 计算训练图的节点嵌入
        embeddings = None
        if embed_method is not None:
            print("Generating Node2Vec embeddings...")
            embeddings = generate_node2vec_embeddings(G_train, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, p=p, q=q)

        # 预计算结构特征
        all_edges = train_pos_edges + train_neg_edges + test_pos_edges + test_neg_edges
        struct_feat_dict = precompute_structural_features(G_train, all_edges, structural_features)
        
        # 构建训练数据
        X_train, y_train = [], []
        for (u, v), label in tqdm([(e, 1) for e in train_pos_edges] + [(e, 0) for e in train_neg_edges], desc="Building training set"):
            feat = struct_feat_dict[(u, v)].copy()
            if embed_method is not None:
                feat = np.concatenate([feat, combine_embeddings(embeddings[u], embeddings[v], method=embed_method)])
            X_train.append(feat)
            y_train.append(label)


        # 构建测试数据
        X_test, y_test = [], []
        for (u, v), label in tqdm([(e, 1) for e in test_pos_edges] + [(e, 0) for e in test_neg_edges], desc="Building test set"):
            feat = struct_feat_dict[(u, v)].copy()
            if embed_method is not None:
                feat = np.concatenate([feat, combine_embeddings(embeddings[u], embeddings[v], method=embed_method)])
            X_test.append(feat)
            y_test.append(label)


        # 转换为 numpy 数组
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        models = {
            'LogisticRegression': LogisticRegression(max_iter=1000,n_jobs=-1),
            'RandomForest': RandomForestClassifier(n_jobs=-1),
            'XGBoost': xgb.XGBClassifier(n_jobs=-1)
        }
        trained_models = {}
        fold_results = {}
        for name, model in models.items():
            model.fit(X_train, y_train)
            trained_models[name] = model
            y_scores = model.predict_proba(X_test)[:, 1]
            metrics = evaluate_metrics(y_test, y_scores)
            print(f"✅ Fold {fold} {name} Metrics: {metrics}")

            # 存储结果
            fold_results[name] = {k: [v] for k, v in metrics.items()}

        # 构建 VotingClassifier 并使用训练好的模型
        voting_clf = VotingClassifier(
            estimators=[
                ('lr', trained_models['LogisticRegression']),
                ('rf', trained_models['RandomForest']),
                ('xgb', trained_models['XGBoost'])
            ],
            voting='soft'
        )
        # 注意 VotingClassifier 仍然需要 fit 一下，用来统一接口
        voting_clf.fit(X_train, y_train)
        y_scores = voting_clf.predict_proba(X_test)[:, 1]
        metrics = evaluate_metrics(y_test, y_scores)
        print(f"✅ Fold {fold} Voting Metrics: {metrics}")
        fold_results['Voting'] = {k: [v] for k, v in metrics.items()}

        results.append(fold_results)

    # 汇总5折平均指标
    average_results = {}
    for model_name in results[0].keys():
        average_results[model_name] = {}
        for metric in results[0][model_name].keys():
            all_scores = []
            for fold in results:
                all_scores.extend(fold[model_name][metric])
            average_results[model_name][metric] = np.mean(all_scores)

    print("\n5折交叉验证结果:")
    for model, scores in average_results.items():
        print(f"\n{model}:")
        for k, v in scores.items():
            print(f"  {k}: {v:.4f}")

    return average_results

# 主函数

In [12]:
######## ！！！！！！参数设置 ！！！！！！#########
dataset_name = "DBLP"  # 可替换为其他数据集名称
# dataset_name = "email-Eu-core"
edges_file_path = f"norm_dataset/{dataset_name}_edges.txt"
G = nx.read_edgelist(edges_file_path, nodetype=int)
structural_features=['HI']  
#structural_features=('CN','JC','AA','RA') # 可根据需要选择特征
# 如果不使用Node2Vec嵌入，embed_method设置为None，那么此时dimensions、walk_length、num_walks、p、q等参数将不生效
# embed_method='hadamard'
embed_method=None
dimensions=64
walk_length=40
num_walks=400
p=3
q=0.3
runs =10 # 设置重复次数


# 主运行
for run in range(runs):
    print(f"\n=== 运行 {run + 1}/{runs} ===")
    # 运行链路预测交叉验证实验
    results = link_prediction_cross_validation(G, structural_features=structural_features, embed_method=embed_method, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, p=p, q=q)

    # ==== 格式化输出 ====
    # 将 results 字典展开为 1 行，列名为 模型_指标，如 Log_acc, Log_f1 等
    row_data = {}
    model_name_map = {
        "LogisticRegression": "Log",
        "RandomForest": "RF",
        "XGBoost": "XGB",
        "Voting": "Voting"
    }

    for model, metrics in results.items():
        short_name = model_name_map.get(model, model)
        for metric, value in metrics.items():
            col_name = f"{short_name}_{metric}"
            row_data[col_name] = round(value, 4)

    # ==== 创建 DataFrame 并写入 ====
    output_dir = f"results/{dataset_name}"
    os.makedirs(output_dir, exist_ok=True)

    # 文件名包含参数信息
    if embed_method:
        param_str = f"emd_{embed_method}_dim_{dimensions}_walk_{walk_length}_num_{num_walks}_p_{p}_q_{q}_{'_'.join(structural_features)}"
    else:
        param_str = f"emb_no_{'_'.join(structural_features)}"
    file_name = f"{dataset_name}_{param_str}.xlsx"
    file_path = os.path.join(output_dir, file_name)

    # 加载已有文件（如果有），否则新建
    if os.path.exists(file_path):
        df = pd.read_excel(file_path)
        df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)
    else:
        df = pd.DataFrame([row_data])

    # 写入 Excel
    df.to_excel(file_path, index=False)
print(f"✅ 实验结果已保存至: {file_path}")


=== 运行 1/10 ===

----- Fold 1/5 -----


Building test set: 100%|██████████| 21148/21148 [00:00<00:00, 641330.52it/s]


✅ Fold 1 LogisticRegression Metrics: {'accuracy': 0.6350955173066011, 'precision': 0.6358277075211562, 'recall': 0.6324002269718176, 'f1': 0.6341093357356218, 'roc_auc': 0.6934084772922108, 'average_precision': 0.70480956002927, 'mcc': 0.2701949603529293}
✅ Fold 1 RandomForest Metrics: {'accuracy': 0.6349063741252128, 'precision': 0.6356891467706649, 'recall': 0.632021940609041, 'f1': 0.6338502394840423, 'roc_auc': 0.6929991291614674, 'average_precision': 0.7034171291187584, 'mcc': 0.2698172380232601}
✅ Fold 1 XGBoost Metrics: {'accuracy': 0.6350955173066011, 'precision': 0.6358277075211562, 'recall': 0.6324002269718176, 'f1': 0.6341093357356218, 'roc_auc': 0.6932991842301451, 'average_precision': 0.704188253434959, 'mcc': 0.2701949603529293}
✅ Fold 1 Voting Metrics: {'accuracy': 0.6350955173066011, 'precision': 0.6358277075211562, 'recall': 0.6324002269718176, 'f1': 0.6341093357356218, 'roc_auc': 0.693374124211082, 'average_precision': 0.7043898686621649, 'mcc': 0.2701949603529293}

-

Building test set: 100%|██████████| 21148/21148 [00:00<00:00, 801571.86it/s]


✅ Fold 2 LogisticRegression Metrics: {'accuracy': 0.6372706638925667, 'precision': 0.6396075791093585, 'recall': 0.6289010781161339, 'f1': 0.6342091459634733, 'roc_auc': 0.6922575641084303, 'average_precision': 0.7067555995745544, 'mcc': 0.2745797990914551}
✅ Fold 2 RandomForest Metrics: {'accuracy': 0.6372706638925667, 'precision': 0.6396075791093585, 'recall': 0.6289010781161339, 'f1': 0.6342091459634733, 'roc_auc': 0.6921577648749557, 'average_precision': 0.7057822727092993, 'mcc': 0.2745797990914551}
✅ Fold 2 XGBoost Metrics: {'accuracy': 0.6372706638925667, 'precision': 0.6396075791093585, 'recall': 0.6289010781161339, 'f1': 0.6342091459634733, 'roc_auc': 0.6922517238163247, 'average_precision': 0.7063139764710035, 'mcc': 0.2745797990914551}
✅ Fold 2 Voting Metrics: {'accuracy': 0.6372706638925667, 'precision': 0.6396075791093585, 'recall': 0.6289010781161339, 'f1': 0.6342091459634733, 'roc_auc': 0.6922372169958116, 'average_precision': 0.7066650848141002, 'mcc': 0.274579799091455

...

✅ Fold 3 LogisticRegression Metrics: {'accuracy': 0.6354393265865885, 'precision': 0.637217324645458, 'recall': 0.6289605599167691, 'f1': 0.6330620210386025, 'roc_auc': 0.6900637856390177, 'average_precision': 0.7046252944417478, 'mcc': 0.27090139598445967}
✅ Fold 3 RandomForest Metrics: {'accuracy': 0.6353920363189256, 'precision': 0.6371825586966938, 'recall': 0.6288659793814433, 'f1': 0.632996953541508, 'roc_auc': 0.6898619443537795, 'average_precision': 0.7032619373409098, 'mcc': 0.27080714065799183}
✅ Fold 3 XGBoost Metrics: {'accuracy': 0.6354393265865885, 'precision': 0.637217324645458, 'recall': 0.6289605599167691, 'f1': 0.6330620210386025, 'roc_auc': 0.6899403604109692, 'average_precision': 0.7036827236523229, 'mcc': 0.27090139598445967}
✅ Fold 3 Voting Metrics: {'accuracy': 0.6354393265865885, 'precision': 0.637217324645458, 'recall': 0.6289605599167691, 'f1': 0.6330620210386025, 'roc_auc': 0.6900134494362108, 'average_precision': 0.7041781030149334, 'mcc': 0.2709013959844596

Building test set: 100%|██████████| 21146/21146 [00:00<00:00, 1001080.76it/s]


✅ Fold 4 LogisticRegression Metrics: {'accuracy': 0.6372363567577792, 'precision': 0.6394387853161637, 'recall': 0.6293388820580724, 'f1': 0.6343486343486343, 'roc_auc': 0.6926763302813123, 'average_precision': 0.7064087372292486, 'mcc': 0.2745069577079529}
✅ Fold 4 RandomForest Metrics: {'accuracy': 0.6369053248841389, 'precision': 0.6391960765458217, 'recall': 0.6286768183107916, 'f1': 0.6338928094602327, 'roc_auc': 0.6919485083277342, 'average_precision': 0.704362423580917, 'mcc': 0.27384773581997257}
✅ Fold 4 XGBoost Metrics: {'accuracy': 0.6370944859547905, 'precision': 0.6393348072671344, 'recall': 0.629055140452095, 'f1': 0.6341533180778032, 'roc_auc': 0.6922818615528281, 'average_precision': 0.7050853886011706, 'mcc': 0.27422442103970035}
✅ Fold 4 Voting Metrics: {'accuracy': 0.6372363567577792, 'precision': 0.6394387853161637, 'recall': 0.6293388820580724, 'f1': 0.6343486343486343, 'roc_auc': 0.6925760514767153, 'average_precision': 0.7061782076963271, 'mcc': 0.274506957707952

Building test set: 100%|██████████| 21146/21146 [00:00<00:00, 978570.67it/s]


✅ Fold 5 LogisticRegression Metrics: {'accuracy': 0.6396954506762508, 'precision': 0.6405060882800608, 'recall': 0.636810744348813, 'f1': 0.6386530709034859, 'roc_auc': 0.6947449004269488, 'average_precision': 0.7055512973190249, 'mcc': 0.2793955513884557}
✅ Fold 5 RandomForest Metrics: {'accuracy': 0.6396954506762508, 'precision': 0.6405060882800608, 'recall': 0.636810744348813, 'f1': 0.6386530709034859, 'roc_auc': 0.6947254261220775, 'average_precision': 0.7059736831684501, 'mcc': 0.2793955513884557}
✅ Fold 5 XGBoost Metrics: {'accuracy': 0.6396954506762508, 'precision': 0.6405060882800608, 'recall': 0.636810744348813, 'f1': 0.6386530709034859, 'roc_auc': 0.6946068538156608, 'average_precision': 0.7057623477119144, 'mcc': 0.2793955513884557}
✅ Fold 5 Voting Metrics: {'accuracy': 0.6396954506762508, 'precision': 0.6405060882800608, 'recall': 0.636810744348813, 'f1': 0.6386530709034859, 'roc_auc': 0.694777685602582, 'average_precision': 0.7062596030603211, 'mcc': 0.2793955513884557}

5

In [1]:
import inspect
from node2vec import Node2Vec

print(inspect.signature(Node2Vec))


(graph, dimensions=128, walk_length=80, num_walks=10, p=1, q=1, weight_key='weight', workers=1, sampling_strategy=None, quiet=False, temp_folder=None)
